In [ ]:
### NOTE: A lot of code was taken from Tut 0! ###

import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv
class Baseline_Data:

    def __init__(self, Filename):
        """load data set"""
        file_dir = "/content/drive/MyDrive/"
        with open(file_dir + Filename,'r') as csvfile:
          data_reader = csv.reader(csvfile)

          data_orig = []
          for row in data_reader:
            data_orig.append(row)

        # loop through field names and populate a dictionary with indices
        fields = {}
        for i in range(len(data_orig[0])):
            fields[data_orig[0][i]] = i

        # exclude the first row when preparing the numpy data structure
        self.data = np.array(data_orig[1:])
        self.fields = fields

    def clean(self, col_header, values = []):
        """Converts column data into numerical values
        (missing data is convereted into nan)"""
        # select the column
        column = self.data[:,self.fields[col_header]]
        # find the ones that are empty and make them nan
        column[column == ''] = np.nan
        # encode the the strings as numbers
        for i in range(len(values)):
            column[column == values[i]] = i
        # overwrite
        self.data[:,self.fields[col_header]] = column

    def keep_columns(self, L):
        """ Select Features """
        feature_data = self.data[:,L]
        feature_data = feature_data.astype(float)
        return feature_data

    def clean_data(self):
        """Converts all data into numerical values
        (missing data is converted into nan)"""
        self.clean('image_id')
        self.clean('n_citi')
        self.clean('bed')
        self.clean('bath')
        self.clean('sqft')
        self.clean('price')

In [ ]:
baselineData = Baseline_Data('baselineData2.csv')
print(baselineData.data[0,:]) # first row of data

baselineData.clean_data()
print(baselineData.data[0,:])

feature_data = baselineData.keep_columns([1, 2, 3, 4, 5]) #get rid of image id
print(feature_data[0,:])

In [ ]:
# replace all nan values with -1 to prevent an error
feature_data[np.isnan(feature_data)==1] = -1

# segment data into training and testing datasets (features)
# We are looking for a 70/30 % split, 70% marker is 8763!
training_data = feature_data[0:8763,0:4]
testing_data = feature_data[8763:,0:4]

# segment data into training and testing labels (targets)
training_labels = feature_data[0:8763,4]
testing_labels = feature_data[8763:,4]

print(len(training_data))
print(training_data)
print(training_labels)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(training_data, training_labels)

c = lr.intercept_

print(c)

In [ ]:
m = lr.coef_
print(m)

In [ ]:
y_pred_train = lr.predict(training_data)

import matplotlib.pyplot as plt
plt.scatter(training_labels, y_pred_train)
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Multiple Linear Regressions House Price Prediction vs Actual Price on Train Set")
plt.show

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
r2_score(training_labels, y_pred_train)

In [ ]:
mean_squared_error(training_labels, y_pred_train)